In [1]:
import pandas as pd
import numpy as np


# Apple stock 10 year analysis


In [2]:

df=pd.read_csv("AAPL.csv")

nasdaq=pd.read_csv("NASDAQ.csv")
df

,Date,Open,High,Low,Close,Adj Close,Volume
0,04/01/2010,7.622500,7.660714,7.585000,7.643214,6.583586,493729600
1,05/01/2010,7.664286,7.699643,7.616071,7.656429,6.594968,601904800
2,06/01/2010,7.656429,7.686786,7.526786,7.534643,6.490066,552160000
3,07/01/2010,7.562500,7.571429,7.466071,7.520714,6.478067,477131200
4,08/01/2010,7.510714,7.571429,7.466429,7.570714,6.521136,447610800
...,...,...,...,...,...,...,...
2510,23/12/2019,70.132500,71.062500,70.092499,71.000000,70.287064,98572000
2511,24/12/2019,71.172501,71.222504,70.730003,71.067497,70.353882,48478800
2512,26/12/2019,71.205002,72.495003,71.175003,72.477501,71.749733,93121200
2513,27/12/2019,72.779999,73.492500,72.029999,72.449997,71.722488,146266000


In [3]:
df.dropna(inplace=True)

In [4]:
df['Returns_n']=0
df['Returns_d']=0
df['Spread_p']=0
for i in range(1,len(df)):
    df.iloc[i,df.columns.get_loc("Returns_n")]=df.iloc[i]['Open']-df.iloc[i-1]['Close'] #Returns_n columns gives return when a stock is bought on previous day night and sold on todays morning
    df.iloc[i,df.columns.get_loc("Returns_d")]=df.iloc[i]['Close']-df.iloc[i]['Open']#Returns_d shows daily return of stock
    df.iloc[i,df.columns.get_loc("Spread_p")]=((df.iloc[i]['High']-df.iloc[i]['Low'])/df.iloc[i]['Low'] )*100 #stock high-low
df

,Date,Open,High,Low,Close,Adj Close,Volume,Returns_n,Returns_d,Spread_p
0,04/01/2010,7.622500,7.660714,7.585000,7.643214,6.583586,493729600,0.000000,0.000000,0.000000
1,05/01/2010,7.664286,7.699643,7.616071,7.656429,6.594968,601904800,0.021072,-0.007857,1.097311
2,06/01/2010,7.656429,7.686786,7.526786,7.534643,6.490066,552160000,0.000000,-0.121786,2.125741
3,07/01/2010,7.562500,7.571429,7.466071,7.520714,6.478067,477131200,0.027857,-0.041786,1.411157
4,08/01/2010,7.510714,7.571429,7.466429,7.570714,6.521136,447610800,-0.010000,0.060000,1.406295
...,...,...,...,...,...,...,...,...,...,...
2510,23/12/2019,70.132500,71.062500,70.092499,71.000000,70.287064,98572000,0.272499,0.867500,1.383887
2511,24/12/2019,71.172501,71.222504,70.730003,71.067497,70.353882,48478800,0.172501,-0.105004,0.696311
2512,26/12/2019,71.205002,72.495003,71.175003,72.477501,71.749733,93121200,0.137505,1.272499,1.854584
2513,27/12/2019,72.779999,73.492500,72.029999,72.449997,71.722488,146266000,0.302498,-0.330002,2.030405


In [5]:
nasdaq.head(2)

,Date,Open,High,Low,Close,Adj Close,Volume
0,03/01/2011,2676.649902,2704.860107,2676.340088,2691.52002,2691.52002,1919660000
1,04/01/2011,2699.860107,2700.879883,2663.639893,2681.25000,2681.25000,2015440000


### Theory 1

In [6]:
#Creating a port dataframe in which if we invest 100 rupes at the start what returns it gives after 9 years and comparing it with NASDAQ index
#investing at closing price only if the daily return for that day is negative
port=pd.DataFrame(df,columns=['Inv'])
port.iloc[0,port.columns.get_loc("Inv")]=100

for i in range(1,len(df)-1):
    if df.iloc[i]['Returns_d'] < 0 :
        port.iloc[i,port.columns.get_loc("Inv")]=df.iloc[i+1]['Returns_n']*port.iloc[i-1]['Inv']/df.iloc[i-1]['Close']+port.iloc[i-1]['Inv']
    else:
        port.iloc[i,port.columns.get_loc("Inv")]=port.iloc[i-1]['Inv']

    
        
#calculating daily return on index NASDAQ

p=pd.DataFrame(nasdaq,columns=['Date','NASDAQ'])
p.iloc[0,p.columns.get_loc("NASDAQ")]=100
for i in range(1,len(nasdaq)):
    p.iloc[i,p.columns.get_loc("NASDAQ")]= ((nasdaq.iloc[i]['Close']-nasdaq.iloc[i]['Open'])*p.iloc[i-1]['NASDAQ']/nasdaq.iloc[i]['Open'])+p.iloc[i-1]['NASDAQ']

final=pd.concat([port,p] ,axis=1,ignore_index=False)
final.set_index(['Date'],inplace=True)
final.dropna(inplace=True)
final

,Inv,NASDAQ
Date,,
03/01/2011,100.000000,100.000000
04/01/2011,100.000000,99.310701
05/01/2011,100.363838,100.361411
06/01/2011,100.230635,100.565891
07/01/2011,100.230635,100.210743
...,...,...
20/12/2019,180.043052,152.841205
23/12/2019,177.081171,152.763509
24/12/2019,178.808166,152.727175


### Theory 2

In [7]:
#Creating a port dataframe in which if we invest 100 rupes at the start what returns it gives after 9 years and comparing it with NASDAQ index
#Investing at closing price and selling at opening next day
port2=pd.DataFrame(df,columns=['Inv'])
port2.iloc[0,port.columns.get_loc("Inv")]=100

for i in range(1,len(df)-1):
    port2.iloc[i,port2.columns.get_loc("Inv")]=df.iloc[i+1]['Returns_n']*port2.iloc[i-1]['Inv']/df.iloc[i-1]['Close']+port2.iloc[i-1]['Inv']
    
    
        
#calculating daily return on index NASDAQ

p1=pd.DataFrame(nasdaq,columns=['Date','NASDAQ'])
p1.iloc[0,p1.columns.get_loc("NASDAQ")]=100
for i in range(1,len(nasdaq)):
    p1.iloc[i,p1.columns.get_loc("NASDAQ")]= ((nasdaq.iloc[i]['Close']-nasdaq.iloc[i]['Open'])*p1.iloc[i-1]['NASDAQ']/nasdaq.iloc[i]['Open'])+p1.iloc[i-1]['NASDAQ']

finale=pd.concat([port2,p1] ,axis=1,ignore_index=False)
finale.set_index(['Date'],inplace=True)
finale.dropna(inplace=True)
finale

,Inv,NASDAQ
Date,,
03/01/2011,100.000000,100.000000
04/01/2011,100.000000,99.310701
05/01/2011,100.363838,100.361411
06/01/2011,100.230635,100.565891
07/01/2011,100.620937,100.210743
...,...,...
20/12/2019,842.454897,152.841205
23/12/2019,828.595706,152.763509
24/12/2019,836.676639,152.727175


Though the number of transactions are increased in second one and it gives a higher return, the brokerage cost is also greatly increased(which is not taken into consideration here)

## Comaprison of change in monthly stock prices due to day and night respectively

In [8]:
df['Date']= pd.to_datetime(df['Date'])
df.set_index("Date",inplace=True)


In [9]:
df1=pd.DataFrame(columns=['Month','Year'])

df1['Returns_n']=df.Returns_n.resample('M').sum()
df1['Returns_d']=df.Returns_d.resample('M').sum()
df1['Close']=df.Close.resample('M').mean()
df1['Open']=df.Open.resample('M').mean()
df1.reset_index(inplace=True)
df1['Month']=df1.Date.dt.month
df1['Year']=df1.Date.dt.year

df1

,Date,Month,Year,Returns_n,Returns_d,Close,Open
0,2010-01-31,1,2010,1.121072,-0.846072,8.111003,8.151293
1,2010-02-28,2,2010,0.528213,0.192501,8.135378,8.124055
2,2010-03-31,3,2010,0.507500,0.424286,8.591071,8.570867
3,2010-04-30,4,2010,0.423214,-0.394284,9.069464,9.086445
4,2010-05-31,5,2010,0.101430,0.417142,8.900964,8.880107
...,...,...,...,...,...,...,...
115,2019-08-31,8,2019,0.787510,2.295002,50.950000,50.850218
116,2019-09-30,9,2019,0.220001,-0.422490,53.810000,53.831125
117,2019-10-31,10,2019,1.979997,3.567497,57.251136,57.088977
118,2019-11-30,11,2019,2.454978,3.852511,59.862387,59.687273


In [13]:
#Considering that the stock price was intially 100 and calulating the change in stock price from Feb 2011 to Dec 2019.
#Just to get an idea whether the stock price increased during thre day or night.

df2=pd.DataFrame(df1,columns=['Month','Year','Price_n','Price_d'])

df2.iloc[0,df2.columns.get_loc("Price_n")]=100
df2.iloc[0,df2.columns.get_loc("Price_d")]=100
for i in range(1,108):
   
    df2.iloc[i,df2.columns.get_loc("Price_n")]=df1.iloc[i]['Returns_n']*df2.iloc[i-1]['Price_n']/df1.iloc[i-1]['Close']+df2.iloc[i-1]['Price_n']
    df2.iloc[i,df2.columns.get_loc("Price_d")]=df1.iloc[i]['Returns_d']*df2.iloc[i-1]['Price_d']/df1.iloc[i]['Open']+df2.iloc[i-1]['Price_d']
    
    
df2.dropna(inplace=True)
df2

,Month,Year,Price_n,Price_d
0,1,2010,100.000000,100.000000
1,2,2010,106.512302,102.369519
2,3,2010,113.156737,107.437145
3,4,2010,118.731072,102.775174
4,5,2010,120.058923,107.603025
...,...,...,...,...
103,8,2018,736.309124,60.949475
104,9,2018,768.022959,62.806482
105,10,2018,748.137570,61.003030
106,11,2018,711.937598,58.673374


Thus it can be safely assumed that the rise in Apple stock price from 7 to 72 dollars is all during the night while there is comparatively no increase during the day. The rise in stock price mostly due to night is an important conclusion.
